# FLDAS Explorer Dashboard
Modified: Jun 17, 2019

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.util import Dynamic
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

# set pandas dataframe float precision 
pd.set_option('display.precision',2)


In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
import utils as u
import hv_utils as  hvu

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
data_dir = Path.home()/'data'
fpath_sa = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
)
fpath_ea = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

---
## Add Callbacks 

### LatLon Tab selector stream

Fetch the lat,lon location from the mouse location

In [ ]:
from holoviews.streams import *#Tap, Selection1D, PointerXY, RangeXY

In [ ]:
# Defne time sereis scatter point selector stream
# tscatter_selection = Selection1D(source=tseries.Scatter.I)

In [ ]:
def cb_rangexy(x_range, y_range):
    lbrt = hvu.ranges2lbrt(x_range, y_range)
    print(f'x_range: {x_range}')
    print(f'y_range: {y_range}')
    print(f'lbrt: ', lbrt)
    
    df = pd.DataFrame( [lbrt], columns='min_x min_y max_x max_y'.split() )
    return hv.Table( df)


---
## Add zonal statistics
Modified: Jun 20, 2019


In [ ]:
box_stream = BoundsXY(bounds=(0,0,0,0),
                      source = timg_ea)


### Add a debugger div to show the current value of the stream 
Automatically the output of the stream is updated as hv.Div element

In [ ]:
def get_debug_div(*args, **kwargs):
    content = f'<p> args: {str(args)} </p>'
    content += """
    <p> kwargs:  </p>
    <ul>
    """
    for k,v in kwargs.items():
        content += f'<li>{k}: {v}</li>'
    content += '</ul>'
    return hv.Div(content)

# Dynamically create the debug div with the stream of interest
dmap_debug = hv.DynamicMap(get_debug_div, streams=[box_stream])

In [ ]:
# timg_ea.opts(width=300, height=300, colorbar=False) + dmap_debug

1. Try with hv.BoundsXY stream

Limitation: handles a single bounding box

In [ ]:
lbtr = box_stream.bounds
print(lbtr)

In [ ]:
def pretty_df_describe(df_describe, stat_types=None):
    """
    Return a new df with stats of kind in 'sel'
    - df_describe: an output dataframe of df.describe() for some dataframe `df`
    """
    if stat_types is None:
        stat_types= ['min', 'max', 'mean', 'std', '25%', '50%', '75%']
    return u.select(df_describe.T, stat_types, axis=1).T

In [ ]:
def bounds_to_div(bounds):
    xmin, ymin, xmax, ymax = map(np.around, bounds)
    return hv.Div(f"""
    <h3>ROI</h3>
    <p>Lat: {(ymin, ymax)}, Lon: {(xmin, xmax)}</p>
    """)

def bounds_to_box(bounds):
    pass

In [ ]:
# Generic functions (without being tied to holoviews functions/elements)
def get_roi_data(xrd, varname, t, bounds):
    """
    Given a dataset, variable name, time index, and (regional) bounds in lat,lon degree,
    compute the distribution of the values within this region
    Args:
    - xrd (xarray.Dataset)
    - varname (str): a data variable
    - t (int or dt.Datetime object)
    - bounds (tuple of floats): in order of lbtr, ie. (xmin, ymin, xmax, ymax)
    - nodata_val (any object): will be used to replace the NaN values 
    
   Returns 
    
    """
    minx, miny, maxx, maxy = bounds
    if isinstance(t, int):
        data = xrd[varname].isel(time=t).sel(X=slice(minx, maxx), Y=slice(miny,maxy))
    else:# isinstance(t):#, np.datetime64):
        data = xrd[varname].sel(time=t, X=slice(minx, maxx), Y=slice(miny,maxy))
#     else:
#         raise ValueError(
#             f"Selected time input should be either int or np.datetime: {type(t)}"
#         )
    return data

def get_roi_stats(xrd, varname, t, bounds):#, nodata_val=0):
    """
    Returns a pd.DataFrame which is an output of a selected roi data's dataframe's describe method
    """
    
    data = get_roi_data(xrd, varname, t, bounds)
    
    # handle nan?
    stats = data.to_dataframe().describe()
    stats = pretty_df_describe(stats)

    return stats

def get_hv_roi_stats(xrd, varname, t, bounds, decimals=None):#nodata_val=0,):
    """
    Returns a holoview's Div element 
    that shows the selected bounds
    """
    data = get_roi_data(xrd, varname, t, bounds)
    df_stats = pretty_df_describe(data.to_dataframe().describe())
    
    if not decimals:
        decimals = 2
    str_bounds = ','.join(map(lambda b: str(np.around(b)), bounds))
    div_stats = hv.Div(np.around(df_stats, decimals=decimals).to_html(), 
                  label=str_bounds)
    return div_stats

# def dyn_hv_roi_hist(xrd, varname, t, bounds):
#     data = get_roi_data(xrd, varname, t, bounds)

#     # Min and Max computed across all time data for this variable
#     gmin = np.nanmin(xrd[varname])
#     gmax = np.nanmax(xrd[varname])
    
#     return hv.operation.histogram(hv.Dataset(data, ['X','Y'], varname),
#                                   bin_range=(gmin,gmax))

In [ ]:

minx,miny,maxx, maxy = box_stream.bounds
roi_stats = get_roi_stats(xrd_ea, varname, 10, lbtr)
display(roi_stats)

In [ ]:
hv_stats = get_hv_roi_stats(xrd_ea, varname, 10, lbtr)
hv_stats

In [ ]:
timg_ea #<< dyn_hv_roi_hist(xrd_ea, varname,10,lbtr) 

In [ ]:
dmap_bounds = hv.DynamicMap(bounds_to_div, streams=[box_stream])
# dmap_roi_stats = hv.DynamicMap(
#     lambda bounds: hv.Div(get_roi_stats(xrd_ea, varname, 10, bounds).to_html()),
#     streams=[box_stream])
dmap_roi_stats = hv.DynamicMap(
    lambda bounds: get_hv_roi_stats(xrd_ea, varname, 10, bounds),
    streams=[box_stream])
dmap_roi_data = hv.DynamicMap(
    lambda bounds: hv.Dataset(get_roi_data(xrd_ea, varname, 10, bounds), ['X','Y'], varname),
    streams = [box_stream])

gmin = np.nanmin(xrd_ea[varname])
gmax = np.nanmax(xrd_ea[varname])
dmap_roi_hist = hv.operation.histogram(dmap_roi_data, 
                                       bin_range=(gmin,gmax))

(
    timg_ea + dmap_debug +
    (dmap_bounds + dmap_roi_hist) + dmap_roi_stats 
    
).cols(2)

In [ ]:
get_hv_roi_stats(xrd_ea, varnames[7],10,  box_stream.bounds)

Success!

2. Extend the selection capability to multiple boxes using `hv.BoxEdit` linked stream

In [ ]:
polys = gv.Polygons([], crs=ccrs.PlateCarree()) # this will become a scriber of the BoxEdit
# sream defined below, so it will be modified as we create roi regions
boxes_stream = BoxEdit(source=polys)

In [ ]:
# ds = xrd_ea[varname]
xrd=xrd_ea
varname=varnames[7]
time='2019-04-05'
def roi_curves(data):
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([], 'time', varname)})
    curves = {}
    data = zip(data['x0'], data['y0'], data['x1'], data['y1'])
    for i, (xmin, ymin, xmax, ymax) in enumerate(data):
#         pdb.set_trace()

        selection = xrd[varname].sel(X=slice(xmin,xmax), Y=slice(ymin,ymax))
        selection = hv.Dataset(selection, ['X','Y', 'time'], varname)
        
        curves[i] = hv.Curve(selection.aggregate('time', np.mean))
    
#     hline = hv.HLine(time, 'time')
    return hv.NdOverlay(curves)# * hline


In [ ]:
# Let's see what the stream outputs
dmap_debug = hv.DynamicMap(get_debug_div, streams=[boxes_stream])
dmap_roi_curves = hv.DynamicMap(roi_curves, streams=[boxes_stream])

In [ ]:
%%opts Curve {+framewise} [width=500]
# (
#     timg_ea * polys + dmap_debug 
#     + dmap_roi_curves
# ).cols(2)

---
## Putting the streams together


## FLDASExplorer with panel
Modified: Jun 13, 2019

In [ ]:
## debug purpose
def get_debug_div(*args, **kwargs):
    content = f'<p> args: {str(args)} </p>'
    content += """
    <p> kwargs:  </p>
    <ul>
    """
    for k,v in kwargs.items():
        content += f'<li>{k}: {v}</li>'
    content += '</ul>'
    return hv.Div(content)
# Quick helper
def bounds_to_div(bounds):
    xmin, ymin, xmax, ymax = map(np.around, bounds)
    return hv.Div(f"""
    <h3>ROI</h3>
    <p>Lat: {(ymin, ymax)}, Lon: {(xmin, xmax)}</p>
    """)


In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 500; H_IMG = 500
W_PLOT = 300; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )
wmts_opts = dict(width=W_IMG, height=H_IMG)

tbl_opts = dict(width = W_PLOT)

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

In [ ]:
import datetime as dt


trange = list(map(pd.Timestamp, hvd_ea.range('time')))

# time_values = xrd.get_index('time')

basemap.opts(**wmts_opts)

class FLDASExplorer(param.Parameterized):

    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10],objects=varnames)
    time = param.Date(trange[0], bounds=trange)
    time_slider= param.Date(trange[0], bounds=trange)
    time_slider_boxedit = param.Date(trange[0], bounds=trange)
    alpha = param.Number(default=1.0, bounds=(0.0,1.0))
#     cmap = param.ObjectSelector(default='fire', objects=['fire'])


    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self):
        super().__init__()
        self._init_streams()
        self._init_debug_box()
        self._set_dataset()
        self._set_dyn_main()
        self._set_dyn_funcs()

        
    @param.depends('region', 'varname', watch=True)
    def _set_dataset(self):
        """
        Upon the change or region and varname, 
        (re)set the current dataset as well as dataset's min and max 
        across all time points
        """
        print("_set_dataset is called")
        # Min and Max computed across all time data for this variable
        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa

        # set dataset
        self.ds = gv.Dataset(self.xrd[self.varname], ['X','Y','time'], self.varname,
                            crs=ccrs.PlateCarree())
        self.dmap = datashade(
            self.ds.to(gv.Image, ['X','Y'], crs=ccrs.PlateCarree(), dynamic=True),
            **ds_opts)
        self.gmin = np.nanmin(self.xrd[self.varname])
        self.gmax = np.nanmax(self.xrd[self.varname])
        print("\tgmin, gmax: ", self.gmin, self.gmax)
        print("Finished (re)setting self.xrd, self.ds, self.damp, self.gmin, self.gmax")
        
        # set boxedit stream and roi_curves with VLines
        print("Setting up boxedit stream")
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.boxedit_stream.source=self.polys
        self.dmap *= self.polys
        # set empty plot for time vlines
        print("Setting up the empty time plot with current time indices")
        self.time_values = self.xrd.get_index('time')
        self.empty_tplot= hv.Curve( (self.time_values, np.empty(len(self.time_values))), 
                                    'time', self.varname).opts(line_alpha=0.)
        self.vlines = hv.HoloMap({t: self.empty_tplot * hv.VLine(t) for t in self.time_values},
                                kdims='time')
        
        
        
    def _init_streams(self):
        self.tap_stream = Tap(name='tap_latlon', x=35.0, y=0.0)#, source=self.main_plot)
        self.sel_stream = Selection1D()
        self.box_stream = BoundsXY( name='box', bounds=(0.,0.,0.,0.))
        self.boxedit_stream = BoxEdit( name='boxedit')
        self.dummy = Tap(name='dummy', x=0, y=0)
        self.streams = [self.tap_stream, self.sel_stream, self.box_stream, self.boxedit_stream]
        print(f"Created initial stream instances...{len(self.streams)}")

    def _init_debug_box(self):
        print("Initialing a debug box...")
        self.debug_box = hv.Div("""
        <h1 style='color:blue;border:1px solid blue'>Debug Box</h1>
        """)
        self.debug_dmap = hv.DynamicMap(self.get_updated_debug_box, 
                                       streams=self.streams)
        self.debug_boxedit = hv.DynamicMap(get_debug_div, 
                                           streams=[self.boxedit_stream])
        print('Initiated debug_dmap with current streams to debug_box')
        
    def get_updated_debug_box(self, *args, **kwargs):
        hvu.append_to_div(self.debug_box, str(args))
        hvu.append_to_div(self.debug_box, str(kwargs))
        return self.debug_box
    
    

    ################################################################################
    ## Initialize Link streams inbetween static components
    ################################################################################
    @param.depends('region', 'varname', 'time', watch=True)
    def _set_dyn_main(self):
        print("Setting main plot dynamically...")
        print('\tTime: ', self.time, type(self.time))
        dmap_img = hv.DynamicMap(self.get_main_img)
        dyn_img = datashade(Dynamic(dmap_img),**ds_opts)

        self.dyn_main = basemap * dyn_img
    
        # must relink the streams 
        self.dyn_img = dyn_img
        self.tap_stream.source = self.dyn_img
        self.box_stream.source = self.dyn_img

    
    # This is where the main components are declared
    def _set_dyn_funcs(self):
        self.dyn_tseries = hv.DynamicMap(self.cb_tseries, streams=[self.tap_stream] )
        self.dyn_tstats = hv.DynamicMap(self.cb_tstats, streams=[self.tap_stream])
        self.dyn_second_plot = basemap * datashade(hv.DynamicMap(self.cb_second_plot, streams=[self.sel_stream]),
                                          **ds_opts)
        self.dmap_bounds = hv.DynamicMap(lambda bounds: bounds_to_div(bounds),
                                         streams=[self.box_stream])
        self.dmap_roi_stats =  hv.DynamicMap(self.cb_roi_stats, streams=[self.box_stream])
        self.dmap_roi_hist = hv.operation.histogram(
            hv.DynamicMap(self.cb_roi_data, streams=[self.box_stream]),
            bin_range=(self.gmin,self.gmax))
        self.dmap_roi_curves = hv.DynamicMap(self.cb_roi_curves, streams=[self.boxedit_stream])

        
    
    ################################################################################
    ## Dynamic Building Blocks (hv.DynamicMaps)
    ################################################################################        
    @param.depends('region', 'varname', 'time')#, watch=True)
    def get_main_img(self):
        print("Getting a new main img...")
        print(f'\tTime: {self.time}, Region: {self.region}')
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = hvu.get_img(self.xrd, self.varname, self.time)
        # debug
        
        return img.relabel(self.varname).opts(**img_opts)

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################
    @param.depends('region', 'varname')
    def cb_tseries(self, x, y, method='nearest'):
        """
        Affects:
        - Until the first time this callback  called by the instance, there is no
        self.tscatter attribute 
        - Everytime it is called, self.tscatter is updated to the new scatter plot 
        object that resides in the caller hv.DynamicMap.plot instance
            - The reason to maintain which self.tscatter we use is to 
        """
        print(f'cb_tseries is called. x,y: {x,y, self.region,self.varname}"')
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = self.xrd[self.varname].sel(X=x, Y=y, method=method)
        tseries_label = f"Time Series for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "

        # Time series as Scatter and Curve
        print('\tself.scatter is updated')
        self.tscatter = hv.Scatter(tseries, label=self.varname)
        curve = hv.Curve(tseries)

        # Important! Connect the self.sel_stream to this new tscatter
        print("\tself.sel_stream's source is changed to the new self.tscatter object")
        self.sel_stream.source = self.tscatter

        # Add HLine at mean over time
        mean = hvu.extract_item(tseries.mean())
        mean_line = hv.HLine(mean, label='tseries_mean')
        t_midpoint = pd.Timestamp(hvu.extract_item(tseries.coords['time'][len(tseries)//2]))
        mean_label = hv.Labels([(t_midpoint, mean, f'mean: {mean:.3f}')])

        # Put together to an Overlay
        overlay = (
            curve.opts(alpha=0.5, line_width=2, **curve_opts) 
            * self.tscatter.opts(padding=0.2, size=10, **scatter_opts) 
            * mean_line.opts(color='black', alpha=0.5, line_width=1)
            * mean_label.opts(text_font_size='8pt',text_alpha=0.5)
        ) 
        overlay = overlay.relabel(tseries_label)
#         self.overlay = overlay

        return overlay #vs. self.overlay
    
    @param.depends('region', 'varname')
    def cb_tstats(self, x, y, 
                  method='nearest',show_as_timestamp=True, decimals=3):
        print(f"cb_tstats is called: {x, y, self.region, self.varname}")
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = self.xrd[self.varname].sel(X=x, Y=y, method=method)
        df = hvu.get_stats(tseries, 
                           show_as_timestamp=show_as_timestamp, decimals=decimals)

        # Add metadata on selected latlon point
    #     df['point_idx'] = index[0]#
        df['lat'] = y
        df['lon'] = x

        cols = df.columns.to_list()
        cols = cols[-2:] + cols[:-2]
        df = df[cols]
        label = f"Time Series Stats for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "
        print("\tUpdated self.tstats_table")
        self.tstats_table = hv.Table(df, label=label).opts(**tbl_opts)
        return self.tstats_table
    
    @param.depends('region', 'varname')
    def cb_second_plot(self, index):
        print('Callback from the scatter plot is called. Selected indices: ', index)
        if not index:
            index = [0] #todo: current tindex
        tidx = index[0]
        
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        img = hvu.get_img(self.xrd, self.varname, tidx)
        label = f"{self.varname} on {self.time.strftime('%Y/%m/%d')}"
        return img


    @param.depends('region', 'varname', 'time')
    def cb_roi_data(self, bounds):
        minx, miny, maxx, maxy = bounds
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa 
#         if isinstance(t, int):
#             data = xrd[varname].isel(time=self.time).sel(X=slice(minx, maxx), Y=slice(miny,maxy))
#         else:
#             data = xrd[varname].sel(time=self.time, X=slice(minx, maxx), Y=slice(miny,maxy))
        data = self.xrd[varname].sel(time=self.time, X=slice(minx, maxx), Y=slice(miny,maxy))

        return hv.Dataset(data, ['X','Y'],self.varname)

        
    @param.depends('region', 'varname', 'time')
    def cb_roi_stats(self, bounds):
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        return get_hv_roi_stats(self.xrd, self.varname, self.time, bounds)
    
    @param.depends('region', 'varname')
    def cb_roi_curves(self, data):
        print("ROI-CURVES: need to check if self.ds is what we mean to access")
    #     pdb.set_trace()
        if not data or not any(len(d) for d in data.values()):
            
            return hv.NdOverlay({0: self.empty_tplot})
        curves = {}
        data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
        for i, (x0,x1,y0,y1) in enumerate(data):
            selection = self.ds.select(X=(x0,x1), Y=(y0,y1))
            curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', self.varname)
    #         pdb.set_trace()
        return hv.NdOverlay(curves, label='roi_curves')
    
    
    
    ################################################################################
    ## Build views
    ################################################################################
    def view(self):
        layout = (
            self.dyn_main
            + self.dyn_tstats
            + self.dyn_tseries
            + self.dyn_second_plot
        )

        return layout.opts(
                    opts.Image(**img_opts),
                    opts.Scatter(**scatter_opts),
                    opts.Curve(**curve_opts),
                    opts.Table(**tbl_opts) 
                ).cols(1)

In [ ]:
explorer = FLDASExplorer()
widgets = pn.Param(explorer.param, widgets={
    'time_slider': pn.widgets.DateSlider,
    'time': pn.widgets.DatePicker
})

In [ ]:
pn.panel( 
    (explorer.dmap 
    + explorer.dmap_roi_curves 
    + explorer.debug_boxedit).cols(2)
)
    

In [ ]:
(
    explorer.dyn_main.opts(height=300, width=300, 
                           active_tools=['wheel_zoom', 'box_select']) + 
    explorer.dmap_bounds+ explorer.dmap_roi_stats + 
    explorer.dmap_roi_hist
).cols(2)

In [ ]:
row1 = pn.Row( widgets, explorer.dyn_main, explorer.dyn_second_plot
#               pn.Column (explorer.dyn_tstats, explorer.dyn_tseries )
             )
row2 = pn.Row(pn.Spacer(width=300), explorer.dyn_tseries, explorer.dyn_tstats)
pn.Column(row1, row2)#.servable()


In [ ]:
# app = pn.Row( explorer.param, explorer.view)
# app = pn.Column( explorer.param, 
#                 pn.panel(explorer.build_main_plot),
#                 pn.panel( explorer.build_tseries_plot), 
#                 pn.panel( explorer.build_second_plot),
# #                 pn.panel( explorer.build_tstats_table),
#                )

In [ ]:
# app.servable()


---
## Arrange widgets 
Modified: Jun 19, 2019

In [ ]:
title = """#MINT Interactive Visualization for Spatiotemporal Data"""
instruction = """
This is a demo of an interactive visualization for a large set of spatiotemporal data. 
Here we use the FLDAS NOAH01 daily data in Southern Africa (SA) and Eastern Africa (EA) for the month of April in 2019.
This demo supports a **correct** visualization of a **large** dataset without falling into common visualization [mistakes](#) 
such as overplotting and deceptive colormapping via [Datashader]() and [Colorcet]().


## Instructions

1. Dataset selection

    - dropdown boxes: choose `region`, `variable`, `time`  of interest
    - To specify `time`, a user can either type a specific time to the `Time` input box
    or use the time slider to navigate through multiple time points

2. All maps are responsive to user inputs from the mouse or trackpad.

    - pan or zoom: navigate the map
    - mouse hover or click: get its value or see through multiple time points at a specific (Lat, Lon)
    - drag or box-select: compute zonal statistics in the selected area

3. The time-series plot also supports interactivity

    - pan, zoom, mouse-hover as above
    - click on a point: allows to see the data at the current timepoint over the entire region

## Suggestions?
We would like to know what kind of functionalities users are most interested in.  
Please drop a line for requests, and we'll start working on adding supports for popular requests!

## Questions?
Please feel free to contact haejinso@usc.edu for any questions.
"""

viz_title = """#FLDAS Explorer POC"""
tseries_title = """##Time Series Analysis"""




In [ ]:
W_HEADER = 300 + W_IMG*2
texts = pn.Column( pn.Pane(title, width=W_HEADER),
          pn.Pane(instruction, width=W_HEADER))
viz_title = pn.Pane(viz_title)
tseries_title = pn.Pane(tseries_title)


In [ ]:
# main panel components
header = pn.Column(texts,viz_title)
row1 = pn.Row( widgets, explorer.dyn_main, explorer.dyn_second_plot)
row2 = pn.Row( tseries_title)
row3 = pn.Row(pn.Spacer(width=300), explorer.dyn_tseries, explorer.dyn_tstats)
pn.Column(header,row1, row2, row3).servable()

---
## FLDAS Zonal Statistics App
Modified: Jun 24, 2019


In [ ]:
import datetime as dt


trange = list(map(pd.Timestamp, hvd_ea.range('time')))
# time_values = xrd.get_index('time')

basemap.opts(**wmts_opts)

class FLDASZonal(param.Parameterized):

    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10],objects=varnames)
    time_slider= param.Date(trange[0], bounds=trange)


    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self):
        super().__init__()
        self._init_streams()
        self._init_debug_box()
        self._set_dataset()
        self._init_plots()
#         self._set_dyn_funcs()
    
    def _init_plots(self):
        self.timg = datashade(
            gv.Image(self.xrd[self.varname].sel(time=self.time_slider), 
                     ['X','Y'], crs=ccrs.PlateCarree()),
            **ds_opts)
        self.empty_tplot= hv.Curve( (self.time_values, np.empty(len(self.time_values))), 
                                    'time', self.varname).opts(line_alpha=0.)
        self.vline = self.empty_tplot * hv.VLine(self.time_slider)
        display(self.timg)
        display(self.vline)
        print("Initialied timg with default params")
        
    @param.depends('region', 'varname', watch=True)
    def _set_dataset(self):
        """
        Upon the change or region and varname, 
        (re)set the current dataset as well as dataset's min and max 
        across all time points
        """
        print("_set_dataset is called")
        # Min and Max computed across all time data for this variable
        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa
        self.time_values = self.xrd.get_index('time')

        # set dataset
        self.ds = gv.Dataset(self.xrd[self.varname], ['X','Y','time'], self.varname,
                            crs=ccrs.PlateCarree())

        self.gmin = np.nanmin(self.xrd[self.varname])
        self.gmax = np.nanmax(self.xrd[self.varname])
        print("\tgmin, gmax: ", self.gmin, self.gmax)
        
                def _set_boxedit_canvas(self):
        self.dmap = datashade(
        self.ds.to(gv.Image, ['X','Y'], crs=ccrs.PlateCarree(), dynamic=True),
        **ds_opts)
        self.dyn_timg = self.xrd.sel(time=self.time)
        print("Finished (re)setting self.xrd, self.ds, self.damp, self.gmin, self.gmax")
        
        # set boxedit stream and roi_curves with VLines
        print("Setting up boxedit stream")
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.boxedit_stream.source=self.polys
        self.timg *= self.polys
        # set empty plot for time vlines
        print("Setting up the empty time plot with current time indices")
        self.time_values = self.xrd.get_index('time')
        self.empty_tplot= hv.Curve( (self.time_values, np.empty(len(self.time_values))), 
                                    'time', self.varname).opts(line_alpha=0.)
        self.vlines = hv.HoloMap({t: self.empty_tplot * hv.VLine(t) for t in self.time_values},
                                kdims='time')
        
    def _init_streams(self):
        self.boxedit_stream = BoxEdit( name='boxedit')
        self.streams = [self.boxedit_stream]
        print(f"Created initial stream instances...{len(self.streams)}")

    def _init_debug_box(self):
        print("Initialing a debug box...")
        self.debug_boxedit = hv.DynamicMap(get_debug_div, 
                                           streams=[self.boxedit_stream])
        print('Initiated debug_dmap with current streams to debug_box')
        

    
    ################################################################################
    ## Initialize Link streams inbetween static components
    ###############################################################################

    
    # This is where the main components are declared
    def _set_dyn_funcs(self):
        self.dmap_roi_curves = hv.DynamicMap(self.cb_roi_curves, streams=[self.boxedit_stream])

        
    
    ################################################################################
    ## Dynamic Building Blocks (hv.DynamicMaps)
    ################################################################################        

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################    
    @param.depends('region', 'varname')
    def cb_roi_curves(self, data):
        print("ROI-CURVES: need to check if self.ds is what we mean to access")
    #     pdb.set_trace()
        if not data or not any(len(d) for d in data.values()):
            
            return hv.NdOverlay({0: self.empty_tplot})
        curves = {}
        data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
        for i, (x0,x1,y0,y1) in enumerate(data):
            selection = self.ds.select(X=(x0,x1), Y=(y0,y1))
            curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', self.varname)
    #         pdb.set_trace()
        return hv.NdOverlay(curves, label='roi_curves')
    
    
    
    ################################################################################
    ## Build views
    ################################################################################
    def view(self):
        layout = (
            self.dmap
            + self.dmap_roi_curves
        )

        return layout.opts(
                    opts.Image(**img_opts),
                    opts.Scatter(**scatter_opts),
                    opts.Curve(**curve_opts),
                    opts.Table(**tbl_opts) 
                ).cols(1)

In [ ]:
exp = FLDASZonal()

In [ ]:
(exp.dmap 
 + exp.debug_boxedit
 + exp.dmap_roi_curves.opts(
     opts.Curve(framewise=True,alpha=1.0))
).cols(2)

---
## Simple Zonal Explorer 
Modified: Jun 25, 2019


In [ ]:
trange = list(map(pd.Timestamp, hvd_ea.range('time')))

class ZonalExplorer(param.Parameterized):
    
    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10],objects=varnames)
    time_slider= param.Date(trange[0], bounds=trange)
    
    ################################################################################
    ## Computations
    ################################################################################
    @param.depends('region', 'varname', 'time_slider', watch=True)
    def timg(self):
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        data = xrd[self.varname].sel(time=self.time_slider)
        hvimg = gv.Image(data, ['X','Y'], self.varname,
                         crs=ccrs.PlateCarree(),
                         label=self.varname).opts(**img_opts)
        return hvimg
    


In [ ]:
ze =ZonalExplorer()
pn.Row(ze.param, ze.timg())

In [ ]:
import datetime as dt


trange = list(map(pd.Timestamp, hvd_ea.range('time')))
# time_values = xrd.get_index('time')

basemap.opts(**wmts_opts)

class FLDASExplorer(param.Parameterized):

    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10],objects=varnames)
    time = param.Date(trange[0], bounds=trange)
    time_slider= param.Date(trange[0], bounds=trange)
    time_slider_boxedit = param.Date(trange[0], bounds=trange)
    alpha = param.Number(default=1.0, bounds=(0.0,1.0))
#     cmap = param.ObjectSelector(default='fire', objects=['fire'])


    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self):
        super().__init__()
        self._init_streams()
        self._init_debug_box()
        self._set_dataset()
        self._set_dyn_main()
        self._set_dyn_funcs()

        
    @param.depends('region', 'varname', watch=True)
    def _set_dataset(self):
        """
        Upon the change or region and varname, 
        (re)set the current dataset as well as dataset's min and max 
        across all time points
        """
        print("_set_dataset is called")
        # Min and Max computed across all time data for this variable
        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa

        # set dataset
        self.ds = gv.Dataset(self.xrd[self.varname], ['X','Y','time'], self.varname,
                            crs=ccrs.PlateCarree())
        self.dmap = datashade(
            self.ds.to(gv.Image, ['X','Y'], crs=ccrs.PlateCarree(), dynamic=True),
            **ds_opts)
        self.gmin = np.nanmin(self.xrd[self.varname])
        self.gmax = np.nanmax(self.xrd[self.varname])
        print("\tgmin, gmax: ", self.gmin, self.gmax)
        print("Finished (re)setting self.xrd, self.ds, self.damp, self.gmin, self.gmax")
        
        # set boxedit stream and roi_curves with VLines
        print("Setting up boxedit stream")
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.boxedit_stream.source=self.polys
        self.dmap *= self.polys
        # set empty plot for time vlines
        print("Setting up the empty time plot with current time indices")
        self.time_values = self.xrd.get_index('time')
        self.empty_tplot= hv.Curve( (self.time_values, np.empty(len(self.time_values))), 
                                    'time', self.varname).opts(line_alpha=0.)
        self.vlines = hv.HoloMap({t: self.empty_tplot * hv.VLine(t) for t in self.time_values},
                                kdims='time')
        
        
        
    def _init_streams(self):
        self.tap_stream = Tap(name='tap_latlon', x=35.0, y=0.0)#, source=self.main_plot)
        self.sel_stream = Selection1D()
        self.box_stream = BoundsXY( name='box', bounds=(0.,0.,0.,0.))
        self.boxedit_stream = BoxEdit( name='boxedit')
        self.dummy = Tap(name='dummy', x=0, y=0)
        self.streams = [self.tap_stream, self.sel_stream, self.box_stream, self.boxedit_stream]
        print(f"Created initial stream instances...{len(self.streams)}")

    def _init_debug_box(self):
        print("Initialing a debug box...")
        self.debug_box = hv.Div("""
        <h1 style='color:blue;border:1px solid blue'>Debug Box</h1>
        """)
        self.debug_dmap = hv.DynamicMap(self.get_updated_debug_box, 
                                       streams=self.streams)
        self.debug_boxedit = hv.DynamicMap(get_debug_div, 
                                           streams=[self.boxedit_stream])
        print('Initiated debug_dmap with current streams to debug_box')
        
    def get_updated_debug_box(self, *args, **kwargs):
        hvu.append_to_div(self.debug_box, str(args))
        hvu.append_to_div(self.debug_box, str(kwargs))
        return self.debug_box
    
    

    ################################################################################
    ## Initialize Link streams inbetween static components
    ################################################################################
    @param.depends('region', 'varname', 'time', watch=True)
    def _set_dyn_main(self):
        print("Setting main plot dynamically...")
        print('\tTime: ', self.time, type(self.time))
        dmap_img = hv.DynamicMap(self.get_main_img)
        dyn_img = datashade(Dynamic(dmap_img),**ds_opts)

        self.dyn_main = basemap * dyn_img
    
        # must relink the streams 
        self.dyn_img = dyn_img
        self.tap_stream.source = self.dyn_img
        self.box_stream.source = self.dyn_img

    
    # This is where the main components are declared
    def _set_dyn_funcs(self):
        self.dyn_tseries = hv.DynamicMap(self.cb_tseries, streams=[self.tap_stream] )
        self.dyn_tstats = hv.DynamicMap(self.cb_tstats, streams=[self.tap_stream])
        self.dyn_second_plot = basemap * datashade(hv.DynamicMap(self.cb_second_plot, streams=[self.sel_stream]),
                                          **ds_opts)
        self.dmap_bounds = hv.DynamicMap(lambda bounds: bounds_to_div(bounds),
                                         streams=[self.box_stream])
        self.dmap_roi_stats =  hv.DynamicMap(self.cb_roi_stats, streams=[self.box_stream])
        self.dmap_roi_hist = hv.operation.histogram(
            hv.DynamicMap(self.cb_roi_data, streams=[self.box_stream]),
            bin_range=(self.gmin,self.gmax))
        self.dmap_roi_curves = hv.DynamicMap(self.cb_roi_curves, streams=[self.boxedit_stream])

        
    
    ################################################################################
    ## Dynamic Building Blocks (hv.DynamicMaps)
    ################################################################################        
    @param.depends('region', 'varname', 'time')#, watch=True)
    def get_main_img(self):
        print("Getting a new main img...")
        print(f'\tTime: {self.time}, Region: {self.region}')
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = hvu.get_img(self.xrd, self.varname, self.time)
        # debug
        
        return img.relabel(self.varname).opts(**img_opts)

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################
    @param.depends('region', 'varname')
    def cb_tseries(self, x, y, method='nearest'):
        """
        Affects:
        - Until the first time this callback  called by the instance, there is no
        self.tscatter attribute 
        - Everytime it is called, self.tscatter is updated to the new scatter plot 
        object that resides in the caller hv.DynamicMap.plot instance
            - The reason to maintain which self.tscatter we use is to 
        """
        print(f'cb_tseries is called. x,y: {x,y, self.region,self.varname}"')
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = self.xrd[self.varname].sel(X=x, Y=y, method=method)
        tseries_label = f"Time Series for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "

        # Time series as Scatter and Curve
        print('\tself.scatter is updated')
        self.tscatter = hv.Scatter(tseries, label=self.varname)
        curve = hv.Curve(tseries)

        # Important! Connect the self.sel_stream to this new tscatter
        print("\tself.sel_stream's source is changed to the new self.tscatter object")
        self.sel_stream.source = self.tscatter

        # Add HLine at mean over time
        mean = hvu.extract_item(tseries.mean())
        mean_line = hv.HLine(mean, label='tseries_mean')
        t_midpoint = pd.Timestamp(hvu.extract_item(tseries.coords['time'][len(tseries)//2]))
        mean_label = hv.Labels([(t_midpoint, mean, f'mean: {mean:.3f}')])

        # Put together to an Overlay
        overlay = (
            curve.opts(alpha=0.5, line_width=2, **curve_opts) 
            * self.tscatter.opts(padding=0.2, size=10, **scatter_opts) 
            * mean_line.opts(color='black', alpha=0.5, line_width=1)
            * mean_label.opts(text_font_size='8pt',text_alpha=0.5)
        ) 
        overlay = overlay.relabel(tseries_label)
#         self.overlay = overlay

        return overlay #vs. self.overlay
    
    @param.depends('region', 'varname')
    def cb_tstats(self, x, y, 
                  method='nearest',show_as_timestamp=True, decimals=3):
        print(f"cb_tstats is called: {x, y, self.region, self.varname}")
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = self.xrd[self.varname].sel(X=x, Y=y, method=method)
        df = hvu.get_stats(tseries, 
                           show_as_timestamp=show_as_timestamp, decimals=decimals)

        # Add metadata on selected latlon point
    #     df['point_idx'] = index[0]#
        df['lat'] = y
        df['lon'] = x

        cols = df.columns.to_list()
        cols = cols[-2:] + cols[:-2]
        df = df[cols]
        label = f"Time Series Stats for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "
        print("\tUpdated self.tstats_table")
        self.tstats_table = hv.Table(df, label=label).opts(**tbl_opts)
        return self.tstats_table
    
    @param.depends('region', 'varname')
    def cb_second_plot(self, index):
        print('Callback from the scatter plot is called. Selected indices: ', index)
        if not index:
            index = [0] #todo: current tindex
        tidx = index[0]
        
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        img = hvu.get_img(self.xrd, self.varname, tidx)
        label = f"{self.varname} on {self.time.strftime('%Y/%m/%d')}"
        return img


    @param.depends('region', 'varname', 'time')
    def cb_roi_data(self, bounds):
        minx, miny, maxx, maxy = bounds
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa 
#         if isinstance(t, int):
#             data = xrd[varname].isel(time=self.time).sel(X=slice(minx, maxx), Y=slice(miny,maxy))
#         else:
#             data = xrd[varname].sel(time=self.time, X=slice(minx, maxx), Y=slice(miny,maxy))
        data = self.xrd[varname].sel(time=self.time, X=slice(minx, maxx), Y=slice(miny,maxy))

        return hv.Dataset(data, ['X','Y'],self.varname)

        
    @param.depends('region', 'varname', 'time')
    def cb_roi_stats(self, bounds):
#         xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        return get_hv_roi_stats(self.xrd, self.varname, self.time, bounds)
    
    @param.depends('region', 'varname')
    def cb_roi_curves(self, data):
        print("ROI-CURVES: need to check if self.ds is what we mean to access")
    #     pdb.set_trace()
        if not data or not any(len(d) for d in data.values()):
            
            return hv.NdOverlay({0: self.empty_tplot})
        curves = {}
        data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
        for i, (x0,x1,y0,y1) in enumerate(data):
            selection = self.ds.select(X=(x0,x1), Y=(y0,y1))
            curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', self.varname)
    #         pdb.set_trace()
        return hv.NdOverlay(curves, label='roi_curves')
    
    
    
    ################################################################################
    ## Build views
    ################################################################################
    def view(self):
        layout = (
            self.dyn_main
            + self.dyn_tstats
            + self.dyn_tseries
            + self.dyn_second_plot
        )

        return layout.opts(
                    opts.Image(**img_opts),
                    opts.Scatter(**scatter_opts),
                    opts.Curve(**curve_opts),
                    opts.Table(**tbl_opts) 
                ).cols(1)